In [1]:
import pandas as pd
import numpy as np
import datetime
import time

종목토론방 date tagging

In [2]:
df_board = pd.read_csv('./../Data/boardcomment/boardcomment_kia.csv',dtype={'com_code': np.str,'comment': np.str})
df_date = pd.read_csv('./../Data/주가_휴장일추출.csv')

C:\Users\seon\AppData\Local\Temp\ipykernel_25176\1908212109.py:1: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df = pd.read_csv('./../Data/boardcomment/boardcomment_kia.csv',dtype={'com_code': np.str,'comment': np.str})


In [3]:
df_board.rename(columns={'com_code':'code','comment':'content','decommend':'unlike','recommend':'like','view':'view_count'},inplace=True)
df_board=df_board[['code', 'title', 'content', 'date', 'like', 'unlike','view_count']]

In [4]:
df_board.head()

,code,title,content,date,like,unlike,view_count
0,000270,공매도 이기고 가면 올라갈 수는 있음 ㅋ,어찌하냐?,2022.12.21 14:29,1,0,564
1,000270,기아 다시 9만 가즈아아아아,가즈아 가즈아 애국기아 민노 out,2022.12.21 14:20,5,2,664
2,000270,기관과 외국인 지들끼리 놀아라 하고,개미는 발빼자 개미할배는 발뺀지 오래다,2022.12.21 14:29,1,1,569
3,000270,"앞으로 사재기 없다, 공장 지을 때 참고 하세요.",it's this both of all i 'm very consider\nthe ...,2022.12.21 14:29,1,0,588
4,000270,외국인들,외국인들\n어제 사는 척 개미 꼬시기 해놓고\n오늘 거래량도 없이 매도하는 거 봐라...,2022.12.21 14:36,2,0,598


In [5]:
df_board.shape

(149210, 7)

In [ ]:
df_board.isnull().sum()

In [6]:
df_board.dropna(inplace=True)

In [7]:
df_board.shape

(149208, 7)

In [8]:
# 최소한의 반응이 있는 글들 사용
# 좋아요 싫어요 합이 1 이상
df_board = df_board.loc[(df_board['like']+df_board['unlike'])>=1]
df_board.reset_index(drop=True,inplace=True)
df_board.shape

(110964, 7)

In [9]:
df_board['date'] = df_board['date'].apply(lambda x: pd.to_datetime(x))
df_date = df_date['날짜']
df_date = df_date.apply(lambda x: pd.to_datetime(str(x)))

In [10]:
df_board['content'] = df_board['content'].apply(lambda x : x.replace('\n',''))

In [11]:
idx = np.where((df_board['date']>datetime.datetime(2017, 11, 30, 17, 50))&(df_board['date']<datetime.datetime(2022, 11, 30, 17,50)))

In [12]:
df_board = df_board.iloc[idx]
df_board.reset_index(drop=True,inplace=True)

In [13]:
idx1 = np.where((df_date>=datetime.datetime(2017, 12, 1))&(df_date<=datetime.datetime(2022, 11, 30)))

In [14]:
df_date = df_date[np.min(idx1)-1:np.max(idx1)+2]
df_date.reset_index(drop=True,inplace=True)


In [16]:
def date_category(x):
    for i in range(len(df_date)-1):
        if ((df_date[i] + datetime.timedelta(hours=17,minutes=50)) <= x) & (x<=(df_date[i+1] + datetime.timedelta(hours=17,minutes=49))):
            return df_date[i+1].strftime('%Y-%m-%d')

In [17]:
df_board['date_cate']= df_board['date'].apply(lambda x: date_category(x))

In [23]:
df_board.to_csv('./../Data/boardcomment/pre_comment/comment_kia.csv',encoding='utf-8-sig')

뉴스 date tagging

In [71]:
df_news = pd.read_csv('./../Data/news/news_삼성바이오로직스.csv')
df_date = pd.read_csv('./../Data/주가_휴장일추출.csv')

In [72]:
df_news.head()

,media,date,date_hour,url,content,photourl
0,매일경제,20171203,2017-12-03 17:55:44,https://n.news.naver.com/mnews/article/009/000...,"삼성물산, 겹호재 덩실덩실 내년 영업익 1조 클럽 도전삼성물산이 건설, 상사로 이어...",https://imgnews.pstatic.net/image/009/2017/12/...
1,더벨,20171206,2017-12-06 08:32:29,http://www.thebell.co.kr/free/content/ArticleV...,"한투운용, 공모주펀드 1년 만에 정리한국투자신탁운용이 지난해 설정했던 한국투자e단기...",NaN
2,이데일리,20171204,2017-12-04 09:11:58,https://n.news.naver.com/mnews/article/018/000...,코스피 나흘만에 반등..기관개인 동반 순매수이데일리 최정희 기자 코스피 지수가 나흘...,NaN
3,이데일리,20171204,2017-12-04 13:44:31,https://n.news.naver.com/mnews/article/018/000...,코스피 반등폭 커져..코스닥은 1대 하락이데일리 최정희 기자 코스피 지수가 나흘만에...,NaN
4,헤럴드경제,20171204,2017-12-04 10:03:27,https://n.news.naver.com/mnews/article/016/000...,"삼성바이오로직스, 세계 최대 규모 바이오의약품 제3공장 준공헤럴드경제손인규 기자 삼...",https://imgnews.pstatic.net/image/016/2017/12/...


In [73]:
# media별 date형식 확인
df_news.drop_duplicates(subset=['media'])

,media,date,date_hour,url,content,photourl
0,매일경제,20171203,2017-12-03 17:55:44,https://n.news.naver.com/mnews/article/009/000...,"삼성물산, 겹호재 덩실덩실 내년 영업익 1조 클럽 도전삼성물산이 건설, 상사로 이어...",https://imgnews.pstatic.net/image/009/2017/12/...
1,더벨,20171206,2017-12-06 08:32:29,http://www.thebell.co.kr/free/content/ArticleV...,"한투운용, 공모주펀드 1년 만에 정리한국투자신탁운용이 지난해 설정했던 한국투자e단기...",NaN
2,이데일리,20171204,2017-12-04 09:11:58,https://n.news.naver.com/mnews/article/018/000...,코스피 나흘만에 반등..기관개인 동반 순매수이데일리 최정희 기자 코스피 지수가 나흘...,NaN
4,헤럴드경제,20171204,2017-12-04 10:03:27,https://n.news.naver.com/mnews/article/016/000...,"삼성바이오로직스, 세계 최대 규모 바이오의약품 제3공장 준공헤럴드경제손인규 기자 삼...",https://imgnews.pstatic.net/image/016/2017/12/...
10,서울경제,20171204,2017-12-04 14:43:26,https://n.news.naver.com/mnews/article/011/000...,포미or e족이 한라 웨스턴파크 송도를 주목하는 이유는나 자신을 위한 과감한 투자 ...,https://imgnews.pstatic.net/image/011/2017/12/...
28,연합뉴스,20171207,2017-12-07 10:40:43,https://n.news.naver.com/mnews/article/001/000...,"코스피 중동 불안에 2,460 무너져코스닥 2종합서울연합뉴스 전명훈 기자 코스피가...",https://imgnews.pstatic.net/image/001/2017/12/...
39,연합인포맥스,20171213,2017.12.13 08:30,http://news.einfomax.co.kr/news/articleView.ht...,증시결산 바이오 바통 터치10배 장사도서울연합인포맥스 김경림 기자 올해 국내증시에...,NaN


In [74]:
df_news.isnull().sum()

media           0
date            0
date_hour       0
url             0
content         0
photourl     6954
dtype: int64

In [75]:
# print('null값 제거 전 :', df_news.shape)
# df_news.dropna(subset=['media', 'date', 'date_hour', 'url', 'content'],inplace=True)
# print('null값 제거 후 :', df_news.shape)

In [76]:
df_news.shape

(22646, 6)

In [77]:
df_news['date_hour'] = df_news['date_hour'].apply(lambda x: pd.to_datetime(x))
df_date = df_date['날짜']
df_date = df_date.apply(lambda x: pd.to_datetime(str(x)))

In [78]:
df_date

0      1996-06-25
1      1996-06-26
2      1996-06-27
3      1996-06-28
4      1996-06-29
          ...    
6641   2022-12-09
6642   2022-12-12
6643   2022-12-13
6644   2022-12-14
6645   2022-12-15
Name: 날짜, Length: 6646, dtype: datetime64[ns]

In [79]:
idx = np.where((df_news['date_hour']>datetime.datetime(2017, 11, 30, 17, 50))&(df_news['date_hour']<datetime.datetime(2022, 11, 30, 17,50)))

In [80]:
df_news = df_news.iloc[idx]
df_news.reset_index(drop=True,inplace=True)

In [81]:
idx1 = np.where((df_date>=datetime.datetime(2017, 12, 1))&(df_date<=datetime.datetime(2022, 11, 30)))

In [82]:
df_date = df_date[np.min(idx1)-1:np.max(idx1)+2]
df_date.reset_index(drop=True,inplace=True)
df_date.shape

(1232,)

In [83]:
df_date

0      2017-11-30
1      2017-12-01
2      2017-12-04
3      2017-12-05
4      2017-12-06
          ...    
1227   2022-11-25
1228   2022-11-28
1229   2022-11-29
1230   2022-11-30
1231   2022-12-01
Name: 날짜, Length: 1232, dtype: datetime64[ns]

In [88]:
def date_category(x):
    for i in range(len(df_date)-1):
        if ((df_date[i] + datetime.timedelta(hours=17,minutes=50)) <= x) & (x<=(df_date[i+1] + datetime.timedelta(hours=17,minutes=49))):
            return df_date[i+1].strftime('%Y-%m-%d')

In [89]:
df_news['date_cate']= df_news['date_hour'].apply(lambda x: date_category(x))

In [90]:
df_news.to_csv('./test.csv',encoding='utf-8-sig')